# ---------------------------------------搭建图形化界面--------------------------------------

# 还需要完善按图片选择和格式固定部分

### 要实现的功能：
- 有一个画布，用于显示待分类的图象
- 有一个按钮和输入框用于从文件中选择图片，选择图片后，图片会出现在画布上
- 有一个按钮用于预测，点击就可一预测，并且预测结果出现在下面的框中

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2 as cv
import import_ipynb

import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog
import pickle
import torch
import torchvision
from torchvision import transforms

from model_resnet18 import resnet18

importing Jupyter notebook from model_resnet18.ipynb


In [2]:
"""加载模型"""
model = resnet18(120) # 需要从模型文件中导入定义的模型函数
# 加载模型参数
model.load_state_dict(torch.load('model/best.ckpt')) 
# 将模型转移到GPU中
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# 模型进入验证模式
model.eval()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Residual(
      (fn1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (fn2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual(
      (fn1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (fn2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, mome

In [3]:
"""创建加载图片的函数"""
# 从文件夹选择图象然后再画布上显示图象，图象需要经过裁剪224*224
def load_image():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp")]) # 直接询问打开图片的内置函数
    if file_path:
        img = Image.open(file_path) # 打开图片
        img.thumbnail((480, 480)) # 调整图片大小
        resized_img = ImageTk.PhotoImage(img)
        canvas.image = resized_img # 保持图片引用，防止被垃圾回收
        
        canvas.create_image(canvas.winfo_width() // 2, canvas.winfo_height() // 2, anchor='center', image=resized_img)
        
        
    

In [4]:
"""创建用于预测的函数"""

# 先要定义输入图片的预处理操作
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])]) 

# 加载需要的索引对应标签字典
with open('model/labels_dict.pickle', 'rb') as f:
    class_map = pickle.load(f)
    


file_path = False
def model_predict():
    global file_path
    global result_text
    if file_path:
        # 先对输入图象进行transform操作
        input_img = Image.open(file_path)
        
        input_img = transform(input_img)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_img = input_img.to(device)
        # 增加一个维度用于模型输入
        input_img = input_img.unsqueeze(0)
        # 预测分类结果
        pre = model(input_img)
        # 选择最大概率为分类结果
        output_label = torch.argmax(pre).item()
        # 加载索引和分类结果的字典

        # 导出分类结果
        predicted_label = class_map[output_label]
        # 在输出框显示结果
        result_text.delete("1.0", tk.END)  # 清空文本框
        result_text.insert(tk.END, f"小狗的种类是: {predicted_label}")
    else:
        result_text.delete("1.0", tk.END)  # 清空文本框
        result_text.insert(tk.END, "请先选择图片")


In [9]:
"""创建一个窗口"""
window = tk.Tk()      
window.title("小狗分类")
window.geometry("1000x800")

"""创建一个画布"""
canvas=Canvas(window, bg='white', width=720, height=480)
canvas.grid(column=0, row=0)


"""创建两个按钮和一个输出框"""

# 第一个按钮用于打开和显示选择的图片
load_button = tk.Button(window, text="选择图片", command=load_image)
load_button.grid(column=0, row=3)

# 第二个按钮用于预测
pre_button = tk.Button(window, text="开始分类", command=model_predict)
pre_button.grid(column=0, row=4)

# 第三个输出框用于显示预测的分类结果
result_text = tk.Text(window, height=2, width=40)
result_text.grid(column=0, row=6)


In [10]:
window.mainloop()